# Introdução ao PyTorch

Lucas de Magalhães Araújo   
Mestrando do Instituto de Computação da Unicamp

---
Podemos considerar o PyTorch por dois ângulos diferentes:

 - Como uma ferramenta de diferenciação automática (Autograd);
 - Como um framework de Machine Learning.
 
Nesta apresentação, veremos estes dois aspectos e como eles se relacionam para realizar experimentos de ML.

In [ ]:
# Importa bibliotecas e define funções auxiliares
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch

# Funções auxiliares de visualização
def plot_loss(loss):
    sns.set()
    plt.plot(loss)
    plt.title('Loss')
    plt.show()

def plot_model_and_data(dataset, model, title=''):
    if "Data" in str(type(dataset)):
        print("Tamanho dataset:", len(dataset))
        l = list(dataset)
        x = [p[0].item() for p in l]
        x = torch.tensor(x)
        y = [p[1].item() for p in l]
        y = torch.tensor(y)
    else:
        x = dataset[0]
        y = dataset[1]
    sns.set()
    plt.plot(x.tolist(), model(x).tolist(), label='modelo', color='red')
    plt.scatter(x.tolist(), y.tolist(), label='dataset', color='green')
    plt.title(title)
    plt.legend()
    plt.show()

def plot_dataset(dataset):
    if "Data" in str(type(dataset)):
        print("Tamanho dataset:", len(dataset))
        l = list(dataset)
        x = [p[0].item() for p in l]
        y = [p[1].item() for p in l]
    else:
        x = dataset[0].tolist()
        y = dataset[1].tolist()
    sns.set_style('darkgrid')
    plt.scatter(x, y, color='green')
    plt.title("Dataset")
    plt.show()

def plot_sinc(x, y):
    print("x.grad middle:", x.grad[n_samples//2-2:n_samples//2+3].tolist())
    print("y middle:", y[n_samples//2-2:n_samples//2+3].tolist())
    sns.set()
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.spines['left'].set_position('zero')
    ax.spines['bottom'].set_position('zero')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    plt.plot(x.detach().numpy(), y.detach(), label='y')
    plt.plot(x.detach(), x.grad, label='dy/dx')
    plt.legend()
    plt.show()

---
## Parte 1: Autograd

O paradigma de *diferenciação automática* permite computar o gradiente de funções a partir do seu **grafo computacional**. 
Em PyTorch, operações com tensores formam grafos computacionais dinamicamente. Estes tensores (objetos da classe **Tensors**) são o tipo básico de dados em PyTorch. 

Tensores possuem dois parâmetros convenientes:
 - $\texttt{requires_grad}$, que permite indicar as variáveis que temos interesse em computar o gradiente;
 - $\texttt{device}$, que permite alocar o tensor na CPU ou GPU (quando disponível). Desta maneira, a paralelização da computação do grafo é transparente ao usuário.


### Exemplo: grafo computacional e computação do gradiente por diferenciação automática (backpropagation)

Queremos computar $\dfrac{dy}{dx}$, dado $y = (1.5(2.5 x))^2 = 9.0 x^2$.

Quebrando a expressão em

$$
\begin{equation}
    \begin{cases}
      x = \text{algum escalar} & \\
      m_1 = 2.5 x & \\
      m_2 = 1.5 m_1 & \\
      y = m_2^2 & 
    \end{cases}       
\end{equation}
$$

e aplicando a regra da cadeia, temos

$$
\dfrac{dy}{dx} = \dfrac{dm_1}{dx}.\dfrac{dy}{dm_1} = \dfrac{dm_1}{dx}.\dfrac{dm_2}{dm_1}.\dfrac{dy}{dm_2}
$$

A computação direta (*forward*) é realizada na própria declaração das expressões. A computação do gradiente é realizada pelo métod $\texttt{backward()}$ aplicado ao alvo do grafo (neste caso, a variável $y$).


O grafo computacional com *forward* e *backward* pode ser ilustrado como

![Grafo Computacional](./grafo_computacional.png)

In [ ]:
xval = 2.0

# Tensor que se tem interesse em computar o gradiente
x = torch.tensor(xval, requires_grad=True)

# Expressões parciais
m1 = 2*x
m2 = 1.5*m1
y = m2**2

# Registra função para imprimir valor do gradiente parcial
def print_grad(name):
    def printer(grad):
        print(name, grad.item())
    return printer
x.register_hook(print_grad('dy/dx = dy/dm1 * dm1/dx = dy/dm1 * 2 ='))
m1.register_hook(print_grad('dy/dm1 = dy/dm2 * dm2/dm1 = dy/dm2 * 1.5 = '))
m2.register_hook(print_grad('dy/dm2 = dy/dy * dy/dm2 = dy/dy * 2*m2 = '))
y.register_hook(print_grad('dy/dy = '))

print('x =', x.item())
print('m1 = 2*x =', m1.item())
print('m2 = 1.5*m1 =', m2.item())
print('y = m2**2 =', y.item())
print()
y.backward()

print()
print("dy/dx (analítico) = 18*x =", 18*x.item())

### Exemplo: customizando método backward()

Em muitos casos, a regra da cadeia será suficiente para computar o gradiente dos parâmetros (variáveis) de interesse. Este é o caso das Redes Neurais e Redes de Convolução, por exemplo. Porém, em alguns contextos podemos querer especificar explicitamente a computação do gradiente. Vejamos o caso da função **sinc**, cujo gradiente automático no ponto de singularidade não tem o comportamento que gostaríamos.


$$
\begin{equation}
    \text{sinc}(x) = 
    \begin{cases}
      1&,  x = 0 & \\
      \dfrac{\text{sen}(x)}{x}&,  x \ne 0 & \\
    \end{cases}       
\end{equation}
$$

In [ ]:
n_samples = 1001

def sinc(x):
    return x.sin()/x

x = torch.linspace(-4, 4, n_samples, requires_grad=True)
#y = sinc(x)
y = Sinc.apply(x)
y.backward(torch.ones_like(x))

# Obs: plota derivada a partir dos valores em x.grad
plot_sinc(x, y)

**Customizando gradiente**: 
- extensão da classe $\texttt{torch.autograd.Function}$
- implementação dos métodos estáticos $\texttt{forward()}$ e $\texttt{backward()}$
- estes métodos **não são acessados pelo usuário** mas pelo PyTorch durante a computação do grafo. Usuário computa a operação através do método $\texttt{apply()}$

In [ ]:
class Sinc(torch.autograd.Function):
    @staticmethod
    def forward(context, input_tensor):
        """
        Na passada pelo forward(), recebemos tensor de entrada e devemos
        retornar o resultado da operação.
        O tensor de entrada (e outras variáveis) podem ser salvas no objeto
        'contexto' para ser utilizado no backward().
        """
        sinc = input_tensor.sin() / input_tensor
        if sinc.ndimension() == 0:
            sinc = sinc.expand(1)
        sinc[torch.isnan(sinc)] = 1
        context.save_for_backward(input_tensor, sinc)
        return sinc.view(input_tensor.shape)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient 
        with respect to the output, and we need to compute the current gradient 
        with respect to the output.
        """
        input_tensor, sinc = ctx.saved_tensors
        grad = (input_tensor.cos() / input_tensor) - (sinc / input_tensor)
        grad[torch.isnan(grad)] = 0
        grad = grad.view(input_tensor.shape)
        return grad * grad_output

### Regressão Linear "na unha"

Vamos utilizar este maquinário de diferenciação automática para computar regressão linear implementando a descida de gradiente "manualmente" (sem otimizador).

Ou seja, dado um conjunto de dados que possui distribuição linear, queremos descobrir os parâmetros $a, b$ de um modelo $m$ tal que 

$m = a x + b$

minimiza o erro entre a reta e as amostras do conjunto.

### Relembrando: Stochastic Gradient Descent (genérico)

**Entrada**:
 - conjunto de treino $(X,Y)$;
 - conjunto de pesos $W$ que parametriza um modelo $h$;
 - taxa de aprendizagem $\lambda$;
 - tamanho do batch $m$.
 
**Saída**:
 - pesos atualizados.
 
**Algoritmo**:

**enquanto** não atingir critério de parada:   
&nbsp;&nbsp;&nbsp;&nbsp;    extrair batch com $m$ amostras $((x_1,y_1), …, (x_m,y_m))$ de $(X, Y)$   
&nbsp;&nbsp;&nbsp;&nbsp;    computar função de custo $J(W)=\frac{1}{m}\sum_{i=1}^md(h_W(x_i),y_i)$   
&nbsp;&nbsp;&nbsp;&nbsp;    **para cada** $w \in W$:   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    $w \leftarrow w - \lambda \frac{\partial J}{\partial w}$

**retorna** pesos atualizados

In [ ]:
# Cria Dataset
a_true, b_true = 3, -2
n_samples = 100

def linear_model(x, a, b):
    return a*x + b

x_data = torch.linspace(-5, 5, n_samples)
y_data = linear_model(x_data, a_true, b_true) + torch.tensor(np.random.normal(scale=1, size=n_samples))

# Plota pontos
plot_dataset([x_data, y_data])

In [ ]:
lr = 1e-2
n_epochs = 500

# Inicialização aleatória dos pesos 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
print("Pesos iniciais: a={:.4f}, b={:.4f}".format(a.item(),b.item()))

# Ajuste de curva
train_loss = []
for epoch in range(n_epochs):
    # Predição com 'a' e 'b' atuais
    y_pred = linear_model(x_data, a, b)

    # Loss: MSE
    loss = ((y_data - y_pred)**2).mean()
    train_loss.append(loss.detach().cpu().numpy())

    # Computa gradientes 
    loss.backward()

    # Descida de gradiente
    # Utilizamos no_grad para não recomputar gradientes
    with torch.no_grad():
        a -= lr * a.grad
        b -= lr * b.grad
    
    # Por padrão, o gradiente do PyTorch é cumulativo e cada otimizador
    # se encarrega de zerá-lo no momento correto.
    # Aqui, temos que fazê-lo manualmente.
    a.grad.zero_()
    b.grad.zero_()

    # Visualiza treino a cada 50 épocas
    if epoch%50 == 0:
        print("Época {}: loss={:.4f}".format(epoch, train_loss[epoch]))

print("Pesos finais: a={:.4f}, b={:.4f}, loss={:.4f}".format(a.item(), b.item(), train_loss[-1]))
print("Queríamos encontrar: a={:.4f}, b={:.4f}".format(a_true, b_true))

plot_loss(train_loss)
plot_model_and_data([x_data, y_data], lambda x: linear_model(x, a, b), title='')

---
## Parte 2: Framework de ML

Além do mecanismo de diferenciação automática, PyTorch é um framework para expressar problemas de Machine Learning. A maneira típica de trabalho é expressar o problema em três partes:

 - Representação do conjunto de dados do problema como implementação da classe $\texttt{Dataset}$;
 - Representação do model que irá fazer as predições como implementação da classe $\texttt{Model}$;
 - Loop de treinamento, que otimiza os parâmetros do modelo utilizando otimizador e função de custo.
 
Vejamos a seguir cada componente

### Classe Dataset

Datasets em PyTorchs devem ser criados como sub-classe da classe abstrata $\texttt{torch.utils.data.Dataset}$. Dois métodos precisam ser implementados:

 - $\texttt{__getitem__(index)}$: retorna a amostra de índice $\texttt{index}$. Tipicamente, os conjuntos de dados tem amostras na forma (entrada, alvo). Este método permite acessar amostras pelo uso de índice entre colchetes: $\texttt{dataset[index]}$.
 - $\texttt{__len__()}$: retorna o tamanho do dataset.

In [ ]:
# Uso dos métodos __getitem__ e __len___
l = [-3, 5, 0, 4, 22]

print("l[2]:", l[2])
print("l.__getitem__(2):", l.__getitem__(2))
print("len(l):", len(l))
print("l.__len__():", l.__len__())

In [ ]:
!ls './dataset' | head

In [ ]:
# Exemplo amostra
!cat './dataset/0233.npy'

In [ ]:
from torch.utils.data import Dataset
import os

class LinearData(Dataset):
    def __init__(self, dataset_dir='./dataset'):
        self.files = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir)]
        self.files.sort()

    def __getitem__(self, idx):
        point = np.loadtxt(self.files[idx])
        x, y = torch.tensor(point)
        return x,y 
    
    def __len__(self):
        return len(self.files)
    
dataset = LinearData()

plot_dataset(dataset)

In [ ]:
dataset[13]

### Classe Model

Modelos em PyTorch devem ser criados como sub-classe da classe abstrata $\texttt{torch.nn.Module}$. 

 - Devemos implementar o método $\texttt{forward(input)}$, que deve retornar o resultado da computação do modelo com a entrada $\texttt{input}$
 - Convém que os parâmetros do modelo sejam instanciados como objetos da classe $\texttt{torch.nn.Parameter}$. Estes parâmetros serão os alvos da otimização.

In [ ]:
from torch.nn import Module, Parameter

# Obs: já existe o modelo Linear disponível,
# aqui estamos reimplementando por fins didáticos
class LinearModel(Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.a = Parameter(torch.randn(1, requires_grad=True))
        self.b = Parameter(torch.randn(1, requires_grad=True))

    def forward(self, x):
        return self.a*x + self.b

model = LinearModel()
plot_model_and_data(dataset, model, title='Modelo Pré-Treino')

### Loop básico de treinamento

Instanciados o dataset e modelo, o ciclo básico de treinamento segue a estrutura ilustrada abaixo. 

    # A classe DataLoader automaticamente extrai batches do dataset
    dataloader = DataLoarder(dataset, batch_size)
    
    # Instancia otimizador, inicializado com learning rate, 
    # parâmetros do modelo e outros parâmetros específicos
    # de cada otimizador.
    optimizer = Optimizer(model.parameters(), lr)
    
    # Para cada época
    for epoch in range(n_epochs):
    
        # Para cada época
        for batch in dataloader:
            inputs, targets = batch

            # Zera os gradientes
            optimizer.zero_grad()

            # Predição do modelo atual
            predictions = model(inputs)
            
            # Erro da predição em relação ao alvo
            loss = loss_function(predictions, targets)
            
            # Computa o gradiente dos parâmetros
            loss.backward()
            
            # Atualiza os parâmetros do modelo
            optimizer.step()

### Regressão Linear com Framework PyTorch


In [ ]:
from torch.optim import SGD
from torch.nn import MSELoss
from torch.utils.data import DataLoader

lr = 5e-3
epochs = 50
batch_size = 100

# Cola
a_true = -4
b_true = np.pi

# Loader: se encarrega em criar batches
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Os parâmetros do modelo são as variáveis que o otimizador
# quer otimizar
optimizer = SGD(model.parameters(), lr=lr)
loss_fn = MSELoss()

# Imprime pesos iniciais
a = model.state_dict()['a']
b = model.state_dict()['b']
print("Pesos iniciais: a={:.4f}, b={:.4f}".format(a.item(),b.item()))
print()

# Ajuste de curva
loss_history = []
for i in range(epochs):
    # Ajuste por mini-batches 
    epoch_loss = 0
    for x_batch, y_batch in loader:
        optimizer.zero_grad()
        
        y_pred = model(x_batch)
        loss = loss_fn(y_batch, y_pred)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    epoch_loss /= len(loader)
    loss_history.append(epoch_loss)
    print("\rÉpoca {} de {}. Loss: {:.4f}".format(i+1, epochs, loss_history[-1]), end='')
print()

# Plota estado final e curva de treino
a = model.state_dict()['a']
b = model.state_dict()['b']
print("Pesos finais: a={:.4f}, b={:.4f}. Loss={:.4f}".format(a.item(), b.item(), loss_history[-1]))
print("Queríamos encontrar: a={:.4f}, b={:.4f}".format(a_true, b_true))
plot_model_and_data(dataset, model, title='Modelo Pré-Treino')
plot_loss(loss_history)

In [ ]:
# Loss esperada a partir da distribuição original
x_data = torch.tensor([p[0] for p in dataset])
y_data = torch.tensor([p[1] for p in dataset])

y_pred = linear_model(x_data, a_true, b_true)
loss = loss_fn(y_data, y_pred)
print("Loss esperada: {:.4f}".format(loss.item()))

## Referências

**Autograd**:
 - https://pytorch.org/docs/stable/notes/autograd.html
 - https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html

**Dataset**:
 - https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset
 
**Module**:
 - https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module
 
**Tutorial**:
 - https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e